# Backtrader Qucistart Guide

1. Basic Setup

In [38]:
import matplotlib.pyplot as plt
import backtrader as bt
import datetime
import os.path
import sys

if __name__ == "__main__":
    cerebro = bt.Cerebro()
    
    data = bt.feeds.YahooFinanceCSVData(dataname="BTC-USD.csv",
                                        fromdate=datetime.datetime(2005, 4, 10),
                                        todate=datetime.datetime(2020, 4, 10),
                                        reverse=False)
    cerebro.broker.setcash(100000.0)
    
    print('Starting Portfolio Value : %.2f' %cerebro.broker.getvalue())
    
    cerebro.run()
    
    print("final Portfolio Value: %.2f" %cerebro.broker.getvalue())

Starting Portfolio Value : 100000.00
final Portfolio Value: 100000.00


2. Create a Strategy

In [50]:
class first(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close                                 # Keep a reference to the "close" line in the data[0] dataseries        
        self.order = None                                                    # To keep track of pending orders
        self.buyprice = None
        self.bycomm = None
 

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:                # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f Cost : %.2f, Comm : %.2f' % 
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                
            elif order.issell():
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
    
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None         # Write down: no pending order

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])                           # Simply log the closing price of the series from the reference

        if self.order:                                                        # Check if an order is pending ... if yes, we cannot send a 2nd one
            return

        if not self.position:                                                 # Check if we are in the market
            if self.dataclose[0] < self.dataclose[-1]:                        # Not yet ... we MIGHT BUY if ...
                    if self.dataclose[-1] < self.dataclose[-2]:               # current close less than previous close
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])      # previous close less than the previous close
                        self.order = self.buy()                               # Keep track of the created order to avoid a 2nd order

        else:

            if len(self) >= (self.bar_executed + 5):                         # Already in the market ... we might sell
                self.log('SELL CREATE, %.2f' % self.dataclose[0])            # SELL, SELL, SELL!!! (with all possible default parameters)
                self.order = self.sell()                                     # Keep track of the created order to avoid a 2nd order


        
if __name__ == "__main__":
    cerebro = bt.Cerebro()
    
    cerebro.addstrategy(first) #Add a strategy
    
    data = bt.feeds.YahooFinanceCSVData(dataname="BTC-USD.csv",
                                        fromdate=datetime.datetime(2005, 4, 10),
                                        todate=datetime.datetime(2020, 4, 10),
                                        reverse=False)
    
    cerebro.adddata(data) #add the data feed to cerebro
    
    cerebro.broker.setcash(100000.0) #set up our start cash
    cerebro.broker.setcommission(commission=0.01)     # Set the commission - 0.1% ... divide by 100 to remove the %

    print('Starting Portfolio Value : %.2f' %cerebro.broker.getvalue())
    cerebro.run()
    print("final Portfolio Value: %.2f" %cerebro.broker.getvalue())

Starting Portfolio Value : 100000.00
2015-04-10, Close, 236.07
2015-04-11, Close, 236.55
2015-04-12, Close, 236.15
2015-04-13, Close, 224.59
2015-04-13, BUY CREATE, 224.59
2015-04-14, BUY EXECUTED, 224.76 Cost : 224.76, Comm : 2.25
2015-04-14, Close, 219.16
2015-04-15, Close, 223.83
2015-04-16, Close, 228.57
2015-04-17, Close, 222.88
2015-04-18, Close, 223.36
2015-04-19, Close, 222.60
2015-04-19, SELL CREATE, 222.60
2015-04-20, SELL EXECUTED, Price: 222.61, Cost: 224.76, Comm 2.23
2015-04-20, Close, 224.63
2015-04-21, Close, 235.27
2015-04-22, Close, 234.18
2015-04-23, Close, 236.46
2015-04-24, Close, 231.27
2015-04-25, Close, 226.39
2015-04-25, BUY CREATE, 226.39
2015-04-26, BUY EXECUTED, 226.41 Cost : 226.41, Comm : 2.26
2015-04-26, Close, 219.43
2015-04-27, Close, 229.29
2015-04-28, Close, 225.85
2015-04-29, Close, 225.81
2015-04-30, Close, 236.15
2015-05-01, Close, 232.08
2015-05-01, SELL CREATE, 232.08
2015-05-02, SELL EXECUTED, Price: 232.34, Cost: 226.41, Comm 2.32
2015-05-02, C

3. Customizing the Strategy: Parameters

In [60]:
import datetime 
import os.path  
import sys 
import backtrader as bt

class TestStrategy(bt.Strategy):
    params = (("maperiod", 15),)
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close

        self.order = None
        self.buyprice = None
        self.buycomm = None    
        
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None    
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))        

        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:

            if self.dataclose[0] < self.sma[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()
                
                
if __name__ == "__main__":
    cerebro = bt.Cerebro()
    
    cerebro.addstrategy(TestStrategy) #Add a strategy
    
    data = bt.feeds.YahooFinanceCSVData(dataname="BTC-USD.csv",
                                        fromdate=datetime.datetime(2005, 4, 10),
                                        todate=datetime.datetime(2020, 4, 10),
                                        reverse=False)
    
    cerebro.adddata(data) #add the data feed to cerebro
    cerebro.broker.setcash(100000.0) #set up our start cash
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)     # Add a FixedSize sizer according to the stake [Check]
    cerebro.broker.setcommission(commission=0.01)     # Set the commission - 0.1% ... divide by 100 to remove the %

    print('Starting Portfolio Value : %.2f' %cerebro.broker.getvalue())
    cerebro.run()
    print("final Portfolio Value: %.2f" %cerebro.broker.getvalue())

Starting Portfolio Value : 100000.00
2015-04-24, Close, 231.27
2015-04-24, BUY CREATE, 231.27
2015-04-25, BUY EXECUTED, Price: 231.24, Cost: 2312.40, Comm 23.12
2015-04-25, Close, 226.39
2015-04-25, SELL CREATE, 226.39
2015-04-26, SELL EXECUTED, Price: 226.41, Cost: 2312.40, Comm 22.64
2015-04-26, OPERATION PROFIT, GROSS -48.30, NET -94.07
2015-04-26, Close, 219.43
2015-04-27, Close, 229.29
2015-04-27, BUY CREATE, 229.29
2015-04-28, BUY EXECUTED, Price: 228.97, Cost: 2289.70, Comm 22.90
2015-04-28, Close, 225.85
2015-04-28, SELL CREATE, 225.85
2015-04-29, SELL EXECUTED, Price: 225.59, Cost: 2289.70, Comm 22.56
2015-04-29, OPERATION PROFIT, GROSS -33.80, NET -79.26
2015-04-29, Close, 225.81
2015-04-30, Close, 236.15
2015-04-30, BUY CREATE, 236.15
2015-05-01, BUY EXECUTED, Price: 235.94, Cost: 2359.40, Comm 23.59
2015-05-01, Close, 232.08
2015-05-02, Close, 234.93
2015-05-03, Close, 240.36
2015-05-04, Close, 239.02
2015-05-05, Close, 236.12
2015-05-06, Close, 229.78
2015-05-06, SELL CREA